# 03 - Simulação de Interações (BKT)

Este notebook implementa a **Etapa 3** do pipeline SINKT: geração de interações simuladas usando apenas o modelo BKT.

## Objetivo
Gerar sequências de interações (acertos/erros) para cada estudante baseado nos parâmetros cognitivos e BKT, sem geração de respostas textuais.

## Saída
- `data/output/notebooks/simulacao_interacoes/interactions_bkt.json`: Arquivo JSON contendo todas as interações simuladas (sem respostas LLM)

## Importação de Bibliotecas

In [6]:
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import random
import time

print("✅ Bibliotecas importadas com sucesso")

✅ Bibliotecas importadas com sucesso


## Carregamento de Dados

In [7]:
with open('data/output/notebooks/geracao_perfis/profiles.json', 'r', encoding='utf-8') as f:
    profiles_data = json.load(f)
profiles = profiles_data['profiles']

with open('data/output/notebooks/geracao_estudantes/students.json', 'r', encoding='utf-8') as f:
    students_data = json.load(f)
students = students_data['students']

with open('data/json/questions_graph.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)
questions = questions_data.get('questions', [])

with open('data/json/concepts_graph.json', 'r', encoding='utf-8') as f:
    concepts_data = json.load(f)
concepts = concepts_data.get('concepts', [])

print(f"✅ Dados carregados:")
print(f"  - Perfis: {len(profiles)}")
print(f"  - Estudantes: {len(students)}")
print(f"  - Questões: {len(questions)}")
print(f"  - Conceitos: {len(concepts)}")

✅ Dados carregados:
  - Perfis: 6
  - Estudantes: 100
  - Questões: 680
  - Conceitos: 251


## Configuração de Parâmetros

In [8]:
MIN_INTERACTIONS_PER_STUDENT = 30
MAX_INTERACTIONS_PER_STUDENT = 60
SEED = 42

CHECKPOINT_INTERVAL = 10
CHECKPOINT_FILE = 'data/output/interactions_bkt_checkpoint.json'

ERROR_TYPES = [
    'misconception',
    'careless',
    'slip',
    'incomplete',
    'misunderstanding'
]

np.random.seed(SEED)
random.seed(SEED)

print(f"✅ Configurações:")
print(f"  - Interações por estudante: {MIN_INTERACTIONS_PER_STUDENT}-{MAX_INTERACTIONS_PER_STUDENT}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} estudantes")
print(f"  - Seed: {SEED}")

✅ Configurações:
  - Interações por estudante: 30-60
  - Checkpoint a cada: 10 estudantes
  - Seed: 42


MIN_INTERACTIONS_PER_STUDENT = 30
MAX_INTERACTIONS_PER_STUDENT = 60
SEED = 42

CHECKPOINT_INTERVAL = 10
CHECKPOINT_FILE = 'data/output/notebooks/simulacao_interacoes/interactions_bkt_checkpoint.json'

ERROR_TYPES = [
    'misconception',
    'careless',
    'slip',
    'incomplete',
    'misunderstanding'
]

np.random.seed(SEED)
random.seed(SEED)

print(f"✅ Configurações:")
print(f"  - Interações por estudante: {MIN_INTERACTIONS_PER_STUDENT}-{MAX_INTERACTIONS_PER_STUDENT}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} estudantes")
print(f"  - Seed: {SEED}")

In [9]:
def calculate_response_probability(student_params: Dict, question_difficulty: float) -> float:
    """Calcula probabilidade de resposta correta baseada em BKT com parâmetros cognitivos."""
    mastery = student_params.get('mastery_init_level', 0.5)
    guess = student_params.get('guess', 0.15)
    slip = student_params.get('slip', 0.1)
    
    logic_skill = student_params.get('logic_skill', 0.5)
    reading_skill = student_params.get('reading_skill', 0.5)
    tech_familiarity = student_params.get('tech_familiarity', 0.5)
    learning_consistency = student_params.get('learning_consistency', 0.5)
    
    cognitive_factor = (logic_skill * 0.3 + reading_skill * 0.2 + tech_familiarity * 0.1)
    adjusted_mastery = mastery * (1 - question_difficulty * 0.3) + cognitive_factor * 0.2
    
    if learning_consistency < 0.5:
        noise = np.random.normal(0, (0.5 - learning_consistency) * 0.2)
        adjusted_mastery += noise
    
    prob = adjusted_mastery + (1 - adjusted_mastery) * guess - adjusted_mastery * slip
    return max(0, min(1, prob))

def update_mastery(current_mastery: float, is_correct: bool, learn_rate: float, 
                  memory_capacity: float = 0.5, time_gap: float = 0) -> float:
    """Atualiza o domínio do estudante considerando capacidade de memória."""
    if is_correct:
        current_mastery += (1 - current_mastery) * learn_rate
    else:
        current_mastery *= (1 - learn_rate * 0.5)
    
    if time_gap > 0:
        decay_factor = 1 - (1 - memory_capacity) * 0.1 * min(time_gap / 3600, 1)
        current_mastery *= max(0.3, decay_factor)
    
    return current_mastery

print("✅ Funções BKT definidas")

✅ Funções BKT definidas


In [10]:
interactions = []
start_time = time.time()

for idx, student in enumerate(students, 1):
    student_id = student['id']
    num_interactions = np.random.randint(MIN_INTERACTIONS_PER_STUDENT, MAX_INTERACTIONS_PER_STUDENT + 1)
    
    current_mastery = student.get('mastery_init_level', 0.5)
    last_interaction_time = datetime.now() - timedelta(days=30)
    
    for i in range(num_interactions):
        question = random.choice(questions)
        question_id = question['id']
        question_difficulty = question.get('difficulty', 0.5)
        
        time_gap = (datetime.now() - last_interaction_time).total_seconds()
        
        prob_correct = calculate_response_probability(student, question_difficulty)
        is_correct = np.random.random() < prob_correct
        
        error_type = None
        if not is_correct:
            error_type = random.choice(ERROR_TYPES)
        
        interaction = {
            'student_id': student_id,
            'question_id': question_id,
            'is_correct': is_correct,
            'mastery_before': round(current_mastery, 4),
            'probability': round(prob_correct, 4),
            'error_type': error_type,
            'timestamp': (datetime.now() - timedelta(days=30-i)).isoformat()
        }
        
        current_mastery = update_mastery(
            current_mastery,
            is_correct,
            student.get('learn_rate', 0.1),
            student.get('memory_capacity', 0.5),
            time_gap
        )
        
        interaction['mastery_after'] = round(current_mastery, 4)
        interactions.append(interaction)
        
        last_interaction_time = datetime.now() - timedelta(days=30-i)
    
    if idx % CHECKPOINT_INTERVAL == 0:
        checkpoint_data = {
            'processed_students': idx,
            'total_interactions': len(interactions),
            'interactions': interactions
        }
        os.makedirs(os.path.dirname(CHECKPOINT_FILE), exist_ok=True)
        with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
            json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)
        print(f"Checkpoint: {idx}/{len(students)} estudantes processados ({len(interactions)} interações)")

elapsed_time = time.time() - start_time
print(f"\n✅ Geração de interações concluída!")
print(f"  - Tempo total: {elapsed_time:.2f}s")
print(f"  - Total de interações: {len(interactions)}")
print(f"  - Média por estudante: {len(interactions)/len(students):.1f}")

Checkpoint: 10/100 estudantes processados (471 interações)
Checkpoint: 20/100 estudantes processados (922 interações)
Checkpoint: 30/100 estudantes processados (1400 interações)
Checkpoint: 40/100 estudantes processados (1814 interações)
Checkpoint: 50/100 estudantes processados (2264 interações)
Checkpoint: 60/100 estudantes processados (2701 interações)
Checkpoint: 70/100 estudantes processados (3160 interações)
Checkpoint: 80/100 estudantes processados (3575 interações)
Checkpoint: 90/100 estudantes processados (4036 interações)
Checkpoint: 100/100 estudantes processados (4528 interações)

✅ Geração de interações concluída!
  - Tempo total: 0.20s
  - Total de interações: 4528
  - Média por estudante: 45.3


## Geração de Interações

In [11]:
output_data = {
    "metadata": {
        "description": "Conjunto de interações simuladas usando BKT (sem respostas LLM)",
        "version": "3.0.0",
        "created_at": datetime.now().isoformat(),
        "total_interactions": len(interactions),
        "total_students": len(students),
        "avg_interactions_per_student": len(interactions) / len(students),
        "seed": SEED
    },
    "interactions": interactions
}

output_file = 'data/output/notebooks/simulacao_interacoes/interactions_bkt.json'

output_file = 'data/output/notebooks/simulacao_interacoes/interactions_bkt.json'

In [12]:
output_data = {
    "metadata": {
        "description": "Conjunto de interações simuladas usando BKT (sem respostas LLM)",
        "version": "3.0.0",
        "created_at": datetime.now().isoformat(),
        "total_interactions": len(interactions),
        "total_students": len(students),
        "avg_interactions_per_student": len(interactions) / len(students),
        "seed": SEED
    },
    "interactions": interactions
}

output_file = 'data/output/notebooks/simulacao_interacoes/interactions_bkt.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

if os.path.exists(CHECKPOINT_FILE):
    os.remove(CHECKPOINT_FILE)
    print(f"Checkpoint removido (processamento completo)")

print(f"\n✅ Interações salvas em: {output_file}")
print(f"Total de interações: {len(interactions)}")
print(f"Tamanho do arquivo: {os.path.getsize(output_file) / (1024*1024):.2f} MB")

Checkpoint removido (processamento completo)

✅ Interações salvas em: data/output/interactions_bkt.json
Total de interações: 4528
Tamanho do arquivo: 1.23 MB


## Análise de Métricas

### Estatísticas Gerais

In [13]:
def analyze_general_statistics(interactions: List[Dict]) -> Dict[str, Any]:
    """Calcula estatísticas gerais das interações."""
    total_interactions = len(interactions)
    correct_interactions = sum(1 for i in interactions if i['is_correct'])
    
    stats = {
        'total_interactions': total_interactions,
        'correct_interactions': correct_interactions,
        'incorrect_interactions': total_interactions - correct_interactions,
        'accuracy': correct_interactions / total_interactions if total_interactions > 0 else 0
    }
    
    masteries = [i['mastery_after'] for i in interactions]
    stats['mastery_statistics'] = {
        'mean': np.mean(masteries),
        'median': np.median(masteries),
        'std': np.std(masteries),
        'min': np.min(masteries),
        'max': np.max(masteries)
    }
    
    return stats

general_stats = analyze_general_statistics(interactions)

print("\n✅ Estatísticas Gerais das Interações:\n")
print(f"  Total de Interações: {general_stats['total_interactions']}")
print(f"  Corretas: {general_stats['correct_interactions']} ({general_stats['accuracy']:.1%})")
print(f"  Incorretas: {general_stats['incorrect_interactions']} ({1-general_stats['accuracy']:.1%})")
print(f"\n  Mastery (Domínio):")
print(f"    Média: {general_stats['mastery_statistics']['mean']:.3f}")
print(f"    Mediana: {general_stats['mastery_statistics']['median']:.3f}")
print(f"    Desvio: {general_stats['mastery_statistics']['std']:.3f}")
print(f"    Range: [{general_stats['mastery_statistics']['min']:.3f}, {general_stats['mastery_statistics']['max']:.3f}]")


✅ Estatísticas Gerais das Interações:

  Total de Interações: 4528
  Corretas: 2135 (47.2%)
  Incorretas: 2393 (52.8%)

  Mastery (Domínio):
    Média: 0.367
    Mediana: 0.364
    Desvio: 0.126
    Range: [0.058, 0.720]


### Distribuição de Erros

In [14]:
def analyze_error_distribution(interactions: List[Dict]) -> Dict[str, Any]:
    """Analisa distribuição de tipos de erro."""
    error_distribution = Counter()
    
    for interaction in interactions:
        if interaction['error_type']:
            error_distribution[interaction['error_type']] += 1
    
    total_errors = sum(error_distribution.values())
    
    error_stats = {}
    for error_type, count in error_distribution.items():
        error_stats[error_type] = {
            'count': count,
            'percentage': (count / total_errors * 100) if total_errors > 0 else 0
        }
    
    return {
        'total_errors': total_errors,
        'error_distribution': error_stats
    }

error_analysis = analyze_error_distribution(interactions)

print("\n✅ Análise de Distribuição de Erros:\n")
print(f"  Total de Erros: {error_analysis['total_errors']}")
print(f"\n  Distribuição por Tipo:")
for error_type, stats in error_analysis['error_distribution'].items():
    print(f"    - {error_type}: {stats['count']} ({stats['percentage']:.1f}%)")


✅ Análise de Distribuição de Erros:

  Total de Erros: 2393

  Distribuição por Tipo:
    - misconception: 456 (19.1%)
    - careless: 479 (20.0%)
    - misunderstanding: 495 (20.7%)
    - slip: 504 (21.1%)
    - incomplete: 459 (19.2%)


### Desempenho por Perfil

In [15]:
def analyze_performance_by_profile(interactions: List[Dict], students: List) -> Dict[str, Any]:
    """Analisa desempenho agrupado por perfil cognitivo."""
    profile_data = defaultdict(lambda: {
        'students': [],
        'accuracies': [],
        'masteries': []
    })
    
    student_interactions = defaultdict(list)
    for interaction in interactions:
        student_interactions[interaction['student_id']].append(interaction)
    
    for student in students:
        student_id = student['id']
        profile_id = student['profile_id']
        student_ints = student_interactions.get(student_id, [])
        
        if not student_ints:
            continue
        
        accuracy = sum(1 for i in student_ints if i['is_correct']) / len(student_ints)
        avg_mastery = np.mean([i['mastery_after'] for i in student_ints])
        
        profile_data[profile_id]['students'].append(student_id)
        profile_data[profile_id]['accuracies'].append(accuracy)
        profile_data[profile_id]['masteries'].append(avg_mastery)
    
    profile_stats = {}
    for profile_id, data in profile_data.items():
        if data['accuracies']:
            profile_stats[profile_id] = {
                'num_students': len(data['students']),
                'accuracy': {
                    'mean': np.mean(data['accuracies']),
                    'std': np.std(data['accuracies'])
                },
                'mastery': {
                    'mean': np.mean(data['masteries']),
                    'std': np.std(data['masteries'])
                }
            }
    
    return profile_stats

profile_performance = analyze_performance_by_profile(interactions, students)

print("\n✅ Desempenho por Perfil Cognitivo:\n")
for profile_id, stats in sorted(profile_performance.items()):
    print(f"  {profile_id}:")
    print(f"    Estudantes: {stats['num_students']}")
    print(f"    Acurácia: {stats['accuracy']['mean']:.1%} (±{stats['accuracy']['std']:.1%})")
    print(f"    Domínio: {stats['mastery']['mean']:.3f} (±{stats['mastery']['std']:.3f})")
    print()


✅ Desempenho por Perfil Cognitivo:

  balanced:
    Estudantes: 30
    Acurácia: 51.2% (±9.6%)
    Domínio: 0.360 (±0.045)

  careful:
    Estudantes: 20
    Acurácia: 47.4% (±8.3%)
    Domínio: 0.345 (±0.051)

  intuitive:
    Estudantes: 10
    Acurácia: 43.6% (±8.0%)
    Domínio: 0.343 (±0.032)

  logical:
    Estudantes: 10
    Acurácia: 49.8% (±8.2%)
    Domínio: 0.373 (±0.041)

  quick_learner:
    Estudantes: 20
    Acurácia: 48.3% (±4.9%)
    Domínio: 0.518 (±0.057)

  struggling:
    Estudantes: 10
    Acurácia: 33.4% (±9.4%)
    Domínio: 0.132 (±0.020)



### Correlação entre Parâmetros e Desempenho

In [16]:
def analyze_parameter_correlations(interactions: List[Dict], students: List) -> Dict[str, Any]:
    """Analisa correlação entre parâmetros dos estudantes e seu desempenho."""
    student_interactions = defaultdict(list)
    for interaction in interactions:
        student_interactions[interaction['student_id']].append(interaction)
    
    data = []
    for student in students:
        student_id = student['id']
        student_ints = student_interactions.get(student_id, [])
        if not student_ints:
            continue
        
        accuracy = sum(1 for i in student_ints if i['is_correct']) / len(student_ints)
        avg_mastery = np.mean([i['mastery_after'] for i in student_ints])
        
        data.append({
            'student_id': student_id,
            'accuracy': accuracy,
            'avg_mastery': avg_mastery,
            'learn_rate': student.get('learn_rate', 0),
            'logic_skill': student.get('logic_skill', 0),
            'reading_skill': student.get('reading_skill', 0),
            'tech_familiarity': student.get('tech_familiarity', 0),
            'memory_capacity': student.get('memory_capacity', 0),
            'learning_consistency': student.get('learning_consistency', 0),
            'mastery_init': student.get('mastery_init_level', 0),
            'slip': student.get('slip', 0),
            'guess': student.get('guess', 0)
        })
    
    df = pd.DataFrame(data)
    
    correlations = {}
    for param in ['learn_rate', 'logic_skill', 'reading_skill', 'tech_familiarity',
                  'memory_capacity', 'learning_consistency', 'mastery_init', 'slip', 'guess']:
        if param in df.columns:
            corr_with_accuracy = df[param].corr(df['accuracy'])
            corr_with_mastery = df[param].corr(df['avg_mastery'])
            correlations[param] = {
                'correlation_with_accuracy': round(corr_with_accuracy, 3),
                'correlation_with_mastery': round(corr_with_mastery, 3)
            }
    
    sorted_by_accuracy = sorted(
        correlations.items(),
        key=lambda x: abs(x[1]['correlation_with_accuracy']),
        reverse=True
    )
    
    return {
        'correlations': dict(sorted_by_accuracy),
        'top_5_factors': [f[0] for f in sorted_by_accuracy[:5]]
    }

correlations = analyze_parameter_correlations(interactions, students)

print("\n✅ Correlação entre Parâmetros e Desempenho:\n")
print("  Parâmetro            | Corr. Acurácia | Corr. Domínio")
print("  " + "-" * 60)
for param, corrs in list(correlations['correlations'].items()):
    acc_corr = corrs['correlation_with_accuracy']
    mas_corr = corrs['correlation_with_mastery']
    print(f"  {param:20s} | {acc_corr:>6.3f}         | {mas_corr:>6.3f}")

print(f"\n  Top 5 Fatores Mais Importantes:")
for i, factor in enumerate(correlations['top_5_factors'], 1):
    print(f"    {i}. {factor}")


✅ Correlação entre Parâmetros e Desempenho:

  Parâmetro            | Corr. Acurácia | Corr. Domínio
  ------------------------------------------------------------
  mastery_init         |  0.592         |  0.714
  tech_familiarity     |  0.350         |  0.868
  slip                 | -0.336         | -0.256
  logic_skill          |  0.331         |  0.574
  guess                | -0.269         | -0.485
  memory_capacity      |  0.246         |  0.849
  learning_consistency |  0.196         |  0.565
  learn_rate           |  0.183         |  0.815
  reading_skill        |  0.088         |  0.331

  Top 5 Fatores Mais Importantes:
    1. mastery_init
    2. tech_familiarity
    3. slip
    4. logic_skill
    5. guess


## Resumo da Execução

In [17]:
print("\n" + "="*70)
print("✅ SIMULAÇÃO DE INTERAÇÕES BKT CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n  Arquivo gerado: {output_file}")
print(f"\n  Resumo:")
print(f"    - Total de interações: {len(interactions)}")
print(f"    - Estudantes: {len(students)}")
print(f"    - Acurácia: {general_stats['accuracy']:.1%}")
print(f"\n✅ Próximo passo: Execute o notebook '04_geracao_respostas_llm.ipynb'")
print("="*70)


✅ SIMULAÇÃO DE INTERAÇÕES BKT CONCLUÍDA COM SUCESSO!

  Arquivo gerado: data/output/interactions_bkt.json

  Resumo:
    - Total de interações: 4528
    - Estudantes: 100
    - Acurácia: 47.2%

✅ Próximo passo: Execute o notebook '04_geracao_respostas_llm.ipynb'
